In [1]:
import functools


@functools.cache
def mix(new_value, secret_number):
    return new_value ^ secret_number


@functools.cache
def prune(secret_number):
    return secret_number % 16777216


@functools.cache
def step1(secret_number):
    new_value = secret_number * 64
    secret_number = mix(new_value, secret_number)
    secret_number = prune(secret_number)
    return secret_number


@functools.cache
def step2(secret_number):
    new_value = secret_number // 32
    secret_number = mix(new_value, secret_number)
    secret_number = prune(secret_number)
    return secret_number


@functools.cache
def step3(secret_number):
    new_value = secret_number * 2048
    secret_number = mix(new_value, secret_number)
    secret_number = prune(secret_number)
    return secret_number


@functools.cache
def step(secret_number):
    secret_number = step1(secret_number)
    secret_number = step2(secret_number)
    secret_number = step3(secret_number)
    return secret_number


def forward_n_steps(secret_number, n=2000):
    for _ in range(n):
        secret_number = step(secret_number)
    return secret_number

In [2]:
with open("../data/day22.txt") as f:
    data = f.read()
data = [int(row) for row in data.split("\n")]

In [3]:
import tqdm

In [ ]:
secret_numbers = [forward_n_steps(initial) for initial in tqdm.tqdm(data)]

In [ ]:
sum(secret_numbers)

# Part 2

In [7]:
import numpy as np

In [8]:
def get_all_secret_numbers(secret_number, n=2_000):
    result = [secret_number]
    for _ in range(n - 1):
        secret_number = step(secret_number)
        result.append(secret_number)
    return result


def find_moment_of_sell(price_changes, sequence):
    for i in range(len(price_changes) - len(sequence) + 1):
        if price_changes[i : i + len(sequence)] == sequence:
            return i + len(sequence) - 1
    return None


# data = [1, 2, 3, 2024]
all_secret_numbers = np.array([get_all_secret_numbers(initial) for initial in data])
prices = all_secret_numbers % 10
price_changes = np.diff(prices, 1, 1)  # .tolist()

In [9]:
def get_bananas_per_sequence(price_changes, prices):
    results = {}
    for i in range(4, len(price_changes) + 1):
        sequence = tuple(price_changes[i - 4 : i])
        if sequence not in results:  # First sale counts
            results[sequence] = prices[i]
    return results

In [13]:
bananas_per_sequence = {}
for price_changes_monkey, prices_monkey in zip(price_changes, prices):
    bananas_monkey = get_bananas_per_sequence(price_changes_monkey, prices_monkey)
    for seq, n_bananas in bananas_monkey.items():
        if seq in bananas_per_sequence:
            bananas_per_sequence[seq] += n_bananas
        else:
            bananas_per_sequence[seq] = n_bananas

In [ ]:
max_bananas = max(bananas_per_sequence.values())
print(max_bananas)
for seq, n_bananas in bananas_per_sequence.items():
    if n_bananas == max_bananas:
        print(seq)